# Data Loading and Cleaning for Hotel Reviews
This notebook demonstrates the process of:
1. Loading the dataset.
2. Cleaning the data (e.g., handling missing values).
3. Saving the cleaned data for further analysis.

In [2]:
# Importing necessary libraries
import pandas as pd

print("Libraries loaded successfully.")

Libraries loaded successfully.


## Step 1: Load the Dataset
We will load the hotel reviews dataset from a CSV file. The raw file is located in the `data/raw/reviews.csv`.

In [3]:
# Define file path
file_path = "../data/raw/reviews.csv"

# Load the dataset
data = pd.read_csv(file_path)

# Display the first few rows
data.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


## Step 2: Filtering for AI Analysis

In this step, we prepare the dataset by applying multiple filters to retain only high-quality and relevant reviews. The dataset will be used for AI-based analysis in subsequent steps. Specifically, we will:

1. **Extract and Filter by `overall` Rating**:
   - Extract the `overall` value from the JSON-like `ratings` field (example: {'service': 5.0, 'cleanliness': 5.0, 'overall': 5.0, 'value': 5.0, 'location': 5.0, 'sleep_quality': 5.0, 'rooms': 5.0}).
   - Retain only reviews with an `overall` rating of 2 or 3, as they provide balanced feedback.

2. **Filter Non-Empty `text` Field**:
   - Ensure the `text` field is non-empty and contains meaningful content.

3. **Filter Valid `date_stayed` Field**:
   - Retain rows with a valid `date_stayed` value for potential timeline context.
   - Convert the `date_stayed` field to a standard datetime format.

4. **Prioritize by `num_helpful_votes`**:
   - Include only reviews with `num_helpful_votes` greater than or equal to a defined threshold (e.g., 5) to focus on valuable reviews.

5. **Save the Filtered Dataset**:
   - Write the filtered rows directly to a new CSV file chunk by chunk, ensuring efficient processing of the large dataset.

This step ensures the dataset is clean, structured, and ready for analysis by AI models.

In [6]:
import ast

# Function to extract the `overall` rating
def extract_overall_rating(rating_str):
    """
    Extracts the 'overall' rating from the JSON-like string in the 'ratings' column.
    """
    try:
        rating_dict = ast.literal_eval(rating_str)  # Convert string to dictionary
        return rating_dict.get('overall', None)  # Extract the 'overall' value
    except (ValueError, SyntaxError):
        return None  # Handle parsing errors gracefully


print(extract_overall_rating("{'service': 5.0, 'cleanliness': 5.0, 'overall': 5.0, 'value': 5.0, 'location': 5.0, 'sleep_quality': 5.0, 'rooms': 5.0}"))

5.0


In [7]:
# File paths
input_file = "../data/raw/reviews.csv"
output_file = "../data/processed/hotel_reviews_filtered.csv"

# Parameters
chunk_size = 100000  # Process file in chunks
write_header = True  # Write header for the first chunk only
helpful_votes_threshold = 5  # Minimum number of helpful votes for inclusion

# Process the CSV file chunk by chunk
for chunk in pd.read_csv(input_file, chunksize=chunk_size):
    # Extract the `overall` rating
    chunk['overall'] = chunk['ratings'].apply(lambda x: extract_overall_rating(x) if isinstance(x, str) else None)

    # Filter rows based on `overall` rating
    filtered_chunk = chunk[(chunk['overall'] == 2) | (chunk['overall'] == 3)]

    # Filter rows with non-empty `text`
    filtered_chunk = filtered_chunk.dropna(subset=['text'])
    filtered_chunk = filtered_chunk[filtered_chunk['text'].str.strip() != ""]

    # Filter rows with valid `date_stayed`
    filtered_chunk = filtered_chunk.dropna(subset=['date_stayed'])  # Ensure no null values before parsing

    # Convert `date_stayed` to datetime format (Month Year)
    filtered_chunk['date_stayed'] = pd.to_datetime(
        filtered_chunk['date_stayed'], format='%B %Y', errors='coerce'
    )

    # Drop rows where `date_stayed` could not be parsed
    filtered_chunk = filtered_chunk.dropna(subset=['date_stayed'])

    # Filter rows with `num_helpful_votes` greater than the threshold
    filtered_chunk = filtered_chunk[filtered_chunk['num_helpful_votes'] >= helpful_votes_threshold]

    # Append the filtered chunk to the output CSV
    filtered_chunk.to_csv(output_file, mode='a', header=write_header, index=False)

    # After the first chunk, set header to False
    write_header = False

print(f"Filtered data saved to {output_file}")

Filtered data saved to ../data/processed/hotel_reviews_filtered.csv
